In [6]:
import os
import re
from collections import defaultdict
from typing import Dict, Set, Tuple
import csv


def find_function_usage(directory: str) -> Tuple[Dict[str, Dict[str, int]], Set[str]]:
    """
    Find all exported functions/variables in .ts and .svelte files within a directory
    and track their usage throughout the codebase.

    Returns:
        - A dictionary mapping export names to files and their usage count
        - A set of unused exports
    """
    export_pattern = re.compile(r'''
        export\s+
        (?:
            (?:function|const|let|var|class|interface|type|enum|async\s+function)\s+([a-zA-Z_]\w*)|
            \{([^}]*)\}|
            default\s+([a-zA-Z_]\w*)
        )
    ''', re.VERBOSE)

    # File extensions to scan
    extensions = ('.ts', '.svelte', '.js', '.jsx', '.tsx')

    # To store all export names and their source files
    exports_map = {}  # name -> source file
    all_files = []

    # First pass: Collect all export names
    print("Scanning for exports...")
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(extensions):
                filepath = os.path.join(root, filename)
                relative_path = os.path.relpath(filepath, directory)
                all_files.append((relative_path, filepath))

                try:
                    with open(filepath, 'r', encoding='utf-8') as file:
                        content = file.read()

                        for match in export_pattern.finditer(content):
                            if match.group(1):
                                name = match.group(1)
                                exports_map[name] = relative_path
                                print(f"Found export: {name} in {relative_path}")
                            elif match.group(2):
                                for item in re.split(r',\s*', match.group(2)):
                                    item = item.strip()
                                    if ' as ' in item:
                                        _, export_name = item.split(' as ')
                                        export_name = export_name.strip()
                                    else:
                                        export_name = item
                                    exports_map[export_name] = relative_path
                                    print(f"Found named export: {export_name} in {relative_path}")
                            elif match.group(3):
                                name = match.group(3)
                                exports_map[name] = relative_path
                                print(f"Found default export: {name} in {relative_path}")

                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

    print(f"Found {len(exports_map)} exports")

    # Second pass: Count occurrences of each export
    print("Scanning for usage...")
    usage_by_export = defaultdict(lambda: defaultdict(int))

    for relative_path, filepath in all_files:
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()
                code_without_comments = re.sub(r'//.*?$|/\*[\s\S]*?\*/', '', content, flags=re.MULTILINE)

                for export_name, source_file in exports_map.items():
                    pattern = r'\b' + re.escape(export_name) + r'\b'
                    matches = re.findall(pattern, code_without_comments)
                    if matches:
                        usage_by_export[export_name][relative_path] = len(matches)
        except Exception as e:
            print(f"Error reading {filepath}: {e}")

    # Identify unused exports
    unused_exports = set()
    for export_name, usages in usage_by_export.items():
        source_file = exports_map.get(export_name)
        if not usages or (len(usages) == 1 and source_file in usages):
            unused_exports.add(export_name)

    return usage_by_export, unused_exports, exports_map


def write_results_to_csv(usage_data, unused_exports, exports_map, output_file="export_usage_report.csv"):
    """Write the results to a CSV file."""
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Export Name", "Source File", "Total Occurrences", "Unused?", "Found In Files"])

        sorted_exports = sorted(exports_map.keys(), key=lambda fn: sum(usage_data.get(fn, {}).values()), reverse=True)

        for export_name in sorted_exports:
            source_file = exports_map.get(export_name, "Unknown")
            usages = usage_data.get(export_name, {})
            total_uses = sum(usages.values())
            is_unused = "YES" if export_name in unused_exports else "NO"
            used_in = ", ".join(usages.keys()) if usages else "None"

            writer.writerow([export_name, source_file, total_uses, is_unused, used_in])

    print(f"Results written to {output_file}")

In [7]:
# Get project directory from user (default to current directory)
directory = r"C:\Users\harold.noble\Desktop\RIC\app\frontend"
usage_data, unused_exports, exports_map = find_function_usage(directory)

print(f"\nFound {len(unused_exports)} unused exports:")
for name in sorted(unused_exports):
    source = exports_map.get(name, "Unknown source")
    print(f"- {name} (defined in {source})")

write_results_to_csv(usage_data, unused_exports, exports_map)

Scanning for exports...
Found default export: config in config\svelte.config.js
Found default export: defineConfig in config\vite.config.ts
Found export: APP_NAME in src\lib\constants.ts
Found export: WEBUI_HOSTNAME in src\lib\constants.ts
Found export: WEBUI_BASE_URL in src\lib\constants.ts
Found export: WEBUI_API_BASE_URL in src\lib\constants.ts
Found export: OLLAMA_API_BASE_URL in src\lib\constants.ts
Found export: OPENAI_API_BASE_URL in src\lib\constants.ts
Found export: AUDIO_API_BASE_URL in src\lib\constants.ts
Found export: IMAGES_API_BASE_URL in src\lib\constants.ts
Found export: RETRIEVAL_API_BASE_URL in src\lib\constants.ts
Found export: WEBUI_VERSION in src\lib\constants.ts
Found export: WEBUI_BUILD_HASH in src\lib\constants.ts
Found export: REQUIRED_OLLAMA_VERSION in src\lib\constants.ts
Found export: SUPPORTED_FILE_TYPES in src\lib\constants.ts
Found export: SUPPORTED_FILE_EXTENSIONS in src\lib\constants.ts
Found export: PASTED_TEXT_CHARACTER_LIMIT in src\lib\constants.ts
